In [92]:
import pandas as pd
import numpy as np

In [93]:
df = pd.read_csv(r'D:\Dev\DS_ML\amazon_review_polarity_csv\test.csv', names = ['verdict', 'header', 'review'])

In [94]:
df #verdict 2 == positive, 1 == negative

,verdict,header,review
0,2,Great CD,My lovely Pat has one of the GREAT voices of h...
1,2,One of the best game music soundtracks - for a...,Despite the fact that I have only played a sma...
2,1,Batteries died within a year ...,I bought this charger in Jul 2003 and it worke...
3,2,"works fine, but Maha Energy is better",Check out Maha Energy's website. Their Powerex...
4,2,Great for the non-audiophile,Reviewed quite a bit of the combo players and ...
...,...,...,...
399995,1,Unbelievable- In a Bad Way,We bought this Thomas for our son who is a hug...
399996,1,"Almost Great, Until it Broke...",My son recieved this as a birthday gift 2 mont...
399997,1,Disappointed !!!,"I bought this toy for my son who loves the ""Th..."
399998,2,Classic Jessica Mitford,This is a compilation of a wide range of Mitfo...


In [95]:
df['merged'] = df['header'] + ' ' + df['review']

In [96]:
df = df.drop(['header', 'review'], axis=1)

In [97]:
df['merged'].isna().sum()

24

In [98]:
df['verdict'].isna().sum()

0

In [99]:
df = df.dropna(subset=['merged'], axis=0)

In [100]:
df['merged'].duplicated().sum()

0

In [101]:
data = df[:100][:]

In [102]:
data

,verdict,merged
0,2,Great CD My lovely Pat has one of the GREAT vo...
1,2,One of the best game music soundtracks - for a...
2,1,Batteries died within a year ... I bought this...
3,2,"works fine, but Maha Energy is better Check ou..."
4,2,Great for the non-audiophile Reviewed quite a ...
...,...,...
95,2,The Scarlet Letter a must read for any America...
96,1,The Scarlet Letter: A soap opera Although this...
97,2,This book is Great! One might notice that many...
98,1,I had to read it... I had to read this book fo...


In [103]:
data['verdict'].nunique()

2

In [104]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import torch
from transformers import TrainingArguments, Trainer
from transformers import BertForSequenceClassification, BertTokenizer

In [105]:
tokeniser = BertTokenizer.from_pretrained('bert-base-uncased', cache_dir=r'D:\models')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', cache_dir=r'D:\models')
#model = model.to('cuda')

e:\Review-Insight\review-env\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. P

In [106]:
X = list(data['merged'])
y = list(data['verdict'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train_tokenized = tokeniser(X_train, padding = True, truncation = True, max_length = 512)
X_test_tokenized = tokeniser(X_train, padding = True, truncation = True, max_length = 512)

In [107]:
X_train_tokenized.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [108]:
len(X_train_tokenized['input_ids'])

80

Create torch dataset

In [109]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx], device='cpu') for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx], device='cpu')
        return item

    def __len__(self):
        return len(self.encodings['input_ids'])

In [110]:
train_dataset = Dataset(X_train_tokenized, y_train)
test_dataset = Dataset(X_test_tokenized, y_test)

In [111]:
train_dataset[7]['labels']

tensor(1)

In [112]:
def compute_metrics(p):
    print(type(p))
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred)
    precision = precision_score(y_true=labels, y_pred=pred)
    f1 = f1_score(y_true=labels, y_pred=pred)

    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

In [113]:
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = '1'
os.environ["CUDA_LAUNCH_BLOCKING"] = 'True'

In [114]:
# Define Trainer
args = TrainingArguments(
    output_dir="output",
    num_train_epochs=2,
    per_device_train_batch_size=1

)
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics
)

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
trainer.train()

  0%|          | 0/160 [00:00<?, ?it/s]e:\Review-Insight\review-env\Lib\site-packages\transformers\models\bert\modeling_bert.py:439: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
  1%|          | 1/160 [00:00<01:41,  1.57it/s]

RuntimeError: CUDA error: CUBLAS_STATUS_EXECUTION_FAILED when calling `cublasSgemm( handle, opa, opb, m, n, k, &alpha, a, lda, b, ldb, &beta, c, ldc)`